In [75]:
from bs4 import BeautifulSoup
from requests import get
from splinter import Browser
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [76]:
# Setup splinter
executable_path = {'executable_path' : ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/jordanheise/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


In [77]:
### Mars News
# Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.
newsURL = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

# Load page into browser
browser.visit(newsURL)
# Find tag for first news headline
if browser.is_element_present_by_css('div[class="list_text"]', wait_time=5):
    firstSlide = browser.find_by_css('div[class="list_text"]')[0]
    # Find Title and paragraph of headline
    news_title = firstSlide.find_by_css('div[class="content_title"]').text
    news_p = firstSlide.find_by_css('div[class="article_teaser_body"]').text
else:
    print('Page timed out:' + newsURL)
print(news_title)
print(news_p)

5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover
The symbols, mottos, and small objects added to the agency's newest Mars rover serve a variety of purposes, from functional to decorative.


In [78]:
#####JPL Mars Space Images - Featured Image#####

In [79]:
# Retrieve featured image link
featured_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(featured_image_url)

In [80]:
# HTML Object 
html_image = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html_image, "html.parser")

# Retrieve background-image url from style tag 
image_url  = soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]

# Website Url 
main_url = "https://www.jpl.nasa.gov"

# Concatenate website url with scrapped route
image_url = main_url + image_url

# Display full link to featured image
image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23341-1920x1200.jpg'

In [83]:
## Mars Facts
# Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
mars_facts_url = 'https://space-facts.com/mars/'
browser.visit(mars_facts_url)

facts_df = pd.read_html(mars_facts_url)[0]
facts_df = facts_df.rename(columns = {0:'unit', 1:'value'})

equatorial_diam = facts_df[facts_df['unit'] == 'Equatorial Diameter:']['value'].values[0]
polar_diam = facts_df[facts_df['unit'] == 'Polar Diameter:']['value'].values[0]
mass = facts_df[facts_df['unit'] == 'Mass:']['value'].values[0]
moons = facts_df[facts_df['unit'] == 'Moons:']['value'].values[0]
orbit_distance = facts_df[facts_df['unit'] == 'Orbit Distance:']['value'].values[0]
orbit_period = facts_df[facts_df['unit'] == 'Orbit Period:']['value'].values[0]
surface_temp = facts_df[facts_df['unit'] == 'Surface Temperature:']['value'].values[0]
first_record = facts_df[facts_df['unit'] == 'First Record:']['value'].values[0]
recorded_by = facts_df[facts_df['unit'] == 'Recorded By:']['value'].values[0]

scrape_results = ({
    'equatorial_diam':equatorial_diam,
    'polar_diam':polar_diam,
    'mass':mass,
    'moons':moons,
    'orbit_distance':orbit_distance,
    'orbit_period':orbit_period,
    'surface_temp':surface_temp,
    'first_record':first_record,
    'recorded_by':recorded_by,
    })

# Use Pandas to convert the data to a HTML table string.
html_table = facts_df
html_table = html_table.to_html()
print(html_table)



<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>unit</th>
      <th>value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
      <td>R

In [90]:
# Visit hemispheres website through splinter module 
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)

In [91]:
# HTML Object
html_hemispheres = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html_hemispheres, 'html.parser')

# Retreive all items that contain mars hemispheres information
items = soup.find_all('div', class_='item')

# Create empty list for hemisphere urls 
hemisphere_image_urls = []

# Store the main_ul 
hemispheres_main_url = 'https://astrogeology.usgs.gov'

# Loop through the items previously stored
for i in items: 
    # Store title
    title = i.find('h3').text
    
    # Store link that leads to full image website
    partial_img_url = i.find('a', class_='itemLink product-item')['href']
    
    # Visit the link that contains the full image website 
    browser.visit(hemispheres_main_url + partial_img_url)
    
    # HTML Object of individual hemisphere information website 
    partial_img_html = browser.html
    
    # Parse HTML with Beautiful Soup for every individual hemisphere information website 
    soup = BeautifulSoup( partial_img_html, 'html.parser')
    
    # Retrieve full image source 
    img_url = hemispheres_main_url + soup.find('img', class_='wide-image')['src']
    
    # Append the retreived information into a list of dictionaries 
    hemisphere_image_urls.append({"title" : title, "img_url" : img_url})
    

# Display hemisphere_image_urls
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]